<a href="https://colab.research.google.com/github/TomoharuKurosu/Tomoharu_DS2/blob/main/FE%E6%A4%9C%E8%A8%BA_%E8%A7%A3%E8%AA%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#FE検診
# %% init
from setting import *
import unicodedata
# %% ufd
def key_clnser(df):
    return df.rename(columns={#変更後のデータフレーム: カラム名が変更されたデータフレームが返されます。
        "住民コード":"宛名番号",#住民コード" → "宛名番号": 「住民コード」というカラム名を「宛名番号」に変更します。
        "住民番号":"宛名番号",
        "個人番号":"宛名番号",
    })
def to_onehot(df):
    return (
        pd
        .get_dummies(df.astype(str).replace('nan',np.nan), drop_first=False)
        .rename(columns=lambda c: "_".join(c.split("_")[:-1]) + "=" + c.split("_")[-1] + "_フラグ")
        * 1
    )#df.astype(str): データフレーム df のすべての値を文字列型に変換します。これにより、カテゴリカル変数が適切に処理されます。
#replace('nan', np.nan): 文字列として扱われている「'nan'」を np.nan（NumPyのNaN値）に置き換えます。
#pd.get_dummies(df, drop_first=False): Pandasの get_dummies 関数を使用して、カテゴリカルデータをワンホットエンコーディングします。
# %%
p_in = "../data_clns+/0.母集団.csv"
df_pop = pd.read_csv(p_in)
df_pop

# %%
p_in = "../data_raw/健康かるて_3ファイル/出生～４か月健診.csv"
p_out = "../data_clns+/2.出生～４か月健診.csv"
_raw = (
    # 入力
    pd
    .read_csv(p_in)
    .pipe(key_clnser)#カラム名が統一された「宛名番号」に変更されます。
    .loc[lambda df: df["宛名番号"].isin(df_pop["宛名番号"])]#df_pop に存在する「宛名番号」を持つ行のみが _raw に残ります。
    # 加工
    # 出力
    .pipe(check_df)
)

# %%
multi_choice = {
    "方法": ['方法_A1', '方法_A2', '方法_A3', '方法_A4'],
    "理由": ['理由_A1','理由_A2', '理由_A3', '理由_A4'],
    "生後１か月時の栄養方法": ['生後１か月時の栄養方法_A1', '生後１か月時の栄養方法_A2'],
    "当てはまるもの": ['当てはまるもの_A1', '当てはまるもの_A2', '当てはまるもの_A3', '当てはまるもの_A4'],
    "相談先など解決方法": ['相談先など解決方法_A1','相談先など解決方法_A2', '相談先など解決方法_A3', '相談先など解決方法_A4', '相談先など解決方法_A5']
}#例えば、「方法」カテゴリには ['方法_A1', '方法_A2', '方法_A3', '方法_A4'] の4つの選択肢があります。
_df_multi = (
    # 入力
    _raw
    .set_index("宛名番号")#宛名番号をインデックス（行ラベル）として設定します
    .filter(regex="方法_|理由_|生後１か月時の栄養方法_|当てはまるもの_|相談先など解決方法_")#.filter(regex="方法_|理由_|生後１か月時の栄養方法_|当てはまるもの_|相談先など解決方法_")regex（正規表現）を使って、カラム名に「方法」、「理由」、「生後１か月時の栄養方法」、「当てはまるもの」、「相談先など解決方法」が含まれる列だけを選択します。
    # 加工
    .stack().reset_index()#データフレームを「スタック」して、選択された列を行に展開し、一度に1つの列しかない形に変換します
    .assign(val = lambda df: df["level_1"].str.split("_").str[0] + "=" + df[0])#assignを使って新しい列 val を追加します。この列は、level_1（スタックされた元の列名）の値を_で分割し、最初の部分と=で結合して作成します。さらに、元の値（df[0]）を結合してフラグ形式の文字列を作成します。
    .pivot_table(index="宛名番号", columns="val", aggfunc="size")#ピボットテーブルを作成し、valを列名として使用し、宛名番号ごとに集計します。aggfunc="size"により、各フラグの出現回数をカウントします。
    .fillna(0)
    # 出力
    .rename(columns=lambda c: "出生～４か月健診_" + c + "_フラグ")#.rename で列名を「出生～４か月健診_」+ val + 「_フラグ」という形式に変更。
    .pipe(check_df)
)
# %%
category = ['総合判定', '医師判定', '医師の所見（定頸）', '医師の所見（追視）',
       '医師の所見（股関節脱臼）', '同居家族の喫煙者', 'お父さんは子育てに参加していま', '妊娠中の喫煙',
        '妊娠中の飲酒', '産後１ヶ月の指導・ケア', '育てにくさ感じているか', '相談など解決方法', 'ゆったりとした気分で子どもと過', 'お父さんは育児をしているか',
       '子育てに自信なかったりイライラ', 'この地域で子育てしたいか', '後追いすること知っているか',
        '乳幼児揺さぶられ症候群', '母　喫煙有無', '父　喫煙有無','マタニティーマークを知', '小児救急電話相談',
       '妊娠中働いていましたか', '職場から配慮されたと思うか', 'マークを利用したか', '母　現在の飲酒'
]#各カテゴリは、特定の質問や調査項目に対応していると
_df_cat = (
    # 入力
    _raw
    .set_index("宛名番号")
    [category]#_raw データフレームを「宛名番号」列でインデックスに設定し、指定した category リストに含まれる列を選択します。
    # 加工
    .pipe(to_onehot)#to_onehot 関数は、カテゴリカルデータをバイナリ（0または1）に変換するための関数です。
    # 出力
    .rename(columns=lambda c: "出生～４か月健診_" + c)#.rename で、列名に「出生～４か月健診_」というプレフィックスを付けます。
    .pipe(check_df)
)
# %%
numeric = [
    '出生時体重', '在胎週数', '出生時身長', '出生時頭囲', '出生時胸囲',  '体重', '身長', '胸囲', '妊娠中の喫煙本数'
]#これらは、出生時や妊娠中の身体的な測定値や状況に関する数値データ
_df_num = (
    # 入力
    _raw
    .set_index("宛名番号")
    [numeric]
    # 加工
    # 出力
    .rename(columns=lambda c: "出生～４か月健診_" + c)
    .pipe(check_df)
)
# %%
_df = (
    df_pop
    [["宛名番号", "年度"]]
    .merge(
        _df_multi
        , on="宛名番号"
        , how="left"
    )#df_pop の「宛名番号」に対応する _df_multi のデータが結合されます。
    .merge(
        _df_cat
        , on="宛名番号"
        , how="left"
    )#_df_cat データフレームと「宛名番号」列で再度左外部結合を行います。
    .merge(
        _df_num
        , on="宛名番号"
        , how="left"
    )
)#_df_num データフレームと「宛名番号」列で左外部結合を行います。これで、最終的に _df_num のデータが結合され、最終的なデータフレーム _df が完成します。
_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
p_in = "../data_raw/健康かるて_3ファイル/1歳６か月健診.csv"
_raw = (
    # 入力
    pd
    .read_csv(p_in)
    .pipe(key_clnser)#.pipe(key_clnser) を使って、カラム名を変換します。
    .loc[lambda df: df["宛名番号"].isin(df_pop["宛名番号"])]#.loc[lambda df: df["宛名番号"].isin(df_pop["宛名番号"])] で、df_pop の「宛名番号」列に存在する「宛名番号」のみを抽出します。
    # 出力
    .set_index("宛名番号")#「宛名番号」列をインデックスに設定します。
    .pipe(check_df)
)
# %%
multi_choice = {
    "方法": ['方法_A1', '方法_A2', '方法_A3', '方法_A4'],
    "理由": ['理由_A1','理由_A2', '理由_A3', '理由_A4'],
    "当てはまるもの": ['当てはまるもの_A1', '当てはまるもの_A2', '当てはまるもの_A3', '当てはまるもの_A4'],
    "不正咬合の種類": ['不正咬合の種類_A1', '不正咬合の種類_A2'],
    "その他の異常": ['その他の異常１', 'その他の異常２']
}
#multi_choice.keys() からキーを取得し、各キーに「_」を付加してリストを作成します。
#join メソッドで、これらのキーを「|」で結合して、正規表現パターンを作成します。このパターンは、後で filter メソッドで使用される列の選択に利用されます。
regex = "|".join([k + "_" for k in multi_choice.keys()])
_df_multi = (
    # 入力 _raw データフレームから、正規表現パターンにマッチする列を選択します。これにより、multi_choice のキーに関連する列のみがフィルタリングされます。
    _raw
    .filter(regex=regex)
    # 加工
    .stack().reset_index()
    .assign(val = lambda df: df["level_1"].str.split("_").str[0] + "=" + df[0].astype(str))
    .pivot_table(index="宛名番号", columns="val", aggfunc="size")
    .fillna(0)
    # 出力
    .rename(columns=lambda c: "1歳６か月健診_" + c + "_フラグ")
    .pipe(check_df)
)
# %%
category = ['総合判定', '医師判定',  'う歯罹患型', '軟組織の異常', '不正咬合',
       'この地域で子育てしたいか', '浴室ドアの工夫', '仕上げ磨き',
       'お父さんは育児をしているか', '父　喫煙有無', '予防接種（四種混合）', '予防接種（麻疹風疹混合）', '発達','所見',
        'カリオスタット', '歯科判定', 'フッ素塗布', '歯科指導', '予防接種',
       '保護者による仕上げみがきをして', '同居家族の喫煙者', '指さしで伝えようとすること知っ', '育てにくさを感じているか',
       '相談先など解決方法', 'ゆったりとした気分で子どもと過'
]
_df_cat = (
    # 入力
    _raw
    [category]
    # 加工
    .pipe(to_onehot)
    # 出力
    .rename(columns=lambda c: "1歳６か月健診_" + c)
    .pipe(check_df)
)
# %%
numeric = [
    '体重', '身長', '胸囲', '頭囲', '虫歯の総本数',  '処置歯', '母　喫煙本数'
]
_df_num = (
    # 入力
    _raw
    [numeric]
    # 加工
    # 出力
    .rename(columns=lambda c: "1歳６か月健診_" + c)
    .pipe(check_df)
)
# %%
p_out = '../data_clns+/2.1歳６か月健診.csv'
_df = (
    df_pop
    [["宛名番号", "年度", "学年齢"]]
    .loc[lambda df: df["学年齢"] >= 1]
    .drop(columns="学年齢")
    .merge(
        _df_multi
        , on="宛名番号"
        , how="left"
    )
    .merge(
        _df_cat
        , on="宛名番号"
        , how="left"
    )
    .merge(
        _df_num
        , on="宛名番号"
        , how="left"
    )
)
_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
p_in = "../data_raw/健康かるて_3ファイル/3歳６か月健診.csv"
_raw = (
    # 入力
    pd
    .read_csv(p_in)
    .pipe(key_clnser)
    .loc[lambda df: df["宛名番号"].isin(df_pop["宛名番号"])]
    # 出力
    .set_index("宛名番号")
    .pipe(check_df)
)

# %%
multi_choice = {
    "方法": ['方法_A1', '方法_A2', '方法_A3', '方法_A4'],
    "理由": ['方法理由_A1', '方法理由_A2', '方法理由_A3', '方法理由_A4'],
    "当てはまるもの": ['当てはまるもの_A1', '当てはまるもの_A2', '当てはまるもの_A3', '当てはまるもの_A4'],
    "不正咬合の種類": ['不正咬合の種類_A1', '不正咬合の種類_A2', '不正咬合の種類_A3'],
    '検尿結果': ['検尿結果_A1', '検尿結果_A2', '検尿結果_A3'],
    "その他の異常": ['その他の異常１', 'その他の異常２']
}
regex = "|".join([k + "_" for k in multi_choice.keys()])
_df_multi = (
    # 入力 _raw データフレームから、正規表現パターンにマッチする列を選択します。これにより、multi_choice のキーに関連する列のみがフィルタリングされます。
    _raw
    .filter(regex=regex)
    # 加工
    .stack().reset_index()
    .assign(val = lambda df: df["level_1"].str.split("_").str[0] + "=" + df[0].astype(str))
    .pivot_table(index="宛名番号", columns="val", aggfunc="size")
    .fillna(0)
    # 出力
    .rename(columns=lambda c: "3歳６か月健診_" + c + "_フラグ")
    .pipe(check_df)
)

# %%
category = ['総合判定',
       'う歯罹患型', '医師判定', '軟組織の異常', '不正咬合',
                     '歯科判定', 'フッ素塗布', '歯科指導', '視力アンケート',
       '視力再検査', '聴力アンケート',
                     'お父さんは子育てに参加していま',
       '喫煙者有無', 'かかりつけ医師', '在園状況', '誘われれば遊びに加わること知っ', 'かかりつけ歯科医師',
       'お父さんは育児をしているか', '育てにくさ感じているか', '相談先など解決方法', 'ゆったりとした気分で子供と過ご',
       'この地域で子育てしたいか',
                      '母　喫煙有無', '父　喫煙有無', '発達', '所見'
]
_df_cat = (
    # 入力
    _raw
    [category]
    # 加工
    .pipe(to_onehot)
    # 出力
    .rename(columns=lambda c: "3歳６か月健診_" + c)
    .pipe(check_df)
)

# %%
numeric = [
     '体重', '身長', '頭囲', '虫歯の総本数', '処置歯', '母　喫煙本数', '父　喫煙本数'
]
_df_num = (
    # 入力
    _raw
    [numeric]
    # 加工
    # 出力
    .rename(columns=lambda c: "3歳６か月健診_" + c)
    .pipe(check_df)
)

# %%
# %%shape 関数は、データフレームの形状（行数と列数）を表示し、そのデータフレームをそのまま返す関数
#例えば、データフレームが100行と5列の場合、df.shape は (100, 5) となります。
def shape(df):
    print(df.shape)
    return df

p_out = '../data_clns+/2.3歳６か月健診.csv'
_df = (
    df_pop
    [["宛名番号", "年度", "学年齢"]]
    .loc[lambda df: df["学年齢"] >= 3] #「学年齢」が3以上の行だけを選択します。lambda 関数を使って条件を指定しています。
    .drop(columns="学年齢")
    .pipe(shape)
    .merge(
        _df_multi
        , on="宛名番号"
        , how="left"
    )
    .pipe(shape)
    .merge(
        _df_cat
        , on="宛名番号"
        , how="left"
    )
    .pipe(shape)
    .merge(
        _df_num
        , on="宛名番号"
        , how="left"
    )
    .pipe(shape)
)
_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
p_out = '../data_clns+/2.性別.csv'
_df = (
    pd
    .read_csv("../data_raw/市民室_1ファイル/001_住基データ.csv")
    .pipe(key_clnser)
    .loc[lambda df: df["宛名番号"].isin(df_pop["宛名番号"])]
    .set_index("宛名番号")
    [["性別"]]
    .pipe(to_onehot)
)
_df = (
    df_pop
    [["宛名番号", "年度"]]
    .pipe(shape)
    .merge(
        _df
        , on="宛名番号"
        , how="left"
    )
    .pipe(shape)
)

_df.to_csv(p_out, index=False)
pd.read_csv(p_out)

# %%
